In [125]:
import re
import networkx

file = open('data/16.txt', 'r')
data = file.read()

file.close()

In [126]:
rates = {}
G = networkx.DiGraph()

for line in data.splitlines():
    rate = int(re.findall(r'\d+', line).pop())
    valve, *tunnels = re.findall(r'[A-Z]{2}', line)

    rates[valve] = rate

    for next in tunnels:
        G.add_edge(valve, next)

In [127]:
def get_all_paths(start):
    paths = []
    
    for valve in rates:
        if rates[valve] == 0:
            continue

        path = networkx.shortest_path(G, start, valve)
        
        if len(path) > 1:
            paths.append(path)

    return paths

def get_pressure(route, time):
    flow, timer, pressure = 0, 0, 0
    
    for path in route:
        timer += len(path)
        pressure += flow * len(path)
        flow += rates[path[-1]]
        
    return pressure + flow * (time - timer)

In [128]:
all_paths = {}
all_movements = []
all_pressures = []

for _, (valve, rate) in enumerate(rates.items()):
    all_paths[valve] = get_all_paths(valve)

def find_routes(current_route, start, open_valves, max_length):
    for path in all_paths[start]:
        open_valve = path[-1]

        if open_valve in open_valves:
            continue

        new_route = current_route + [path]

        if len(sum(new_route, [])) > max_length:
            continue

        all_movements.append(set([p[-1] for p in new_route]))
        all_pressures.append(get_pressure(new_route, max_length))
        find_routes(new_route, open_valve, open_valves + [open_valve], max_length)

find_routes([], 'AA', [], 30)

max(all_pressures)

1638

In [129]:
import itertools

all_movements = []
all_pressures = []

find_routes([], 'AA', [], 26)

max_pressure = 0

for p1, p2 in itertools.combinations(range(len(all_movements)), 2):
    pressure = all_pressures[p1] + all_pressures[p2]

    if pressure > max_pressure and len(all_movements[p1] & all_movements[p2]) == 0:
        max_pressure = pressure
    
max_pressure

2400